In this week we are dealing with Credit Risk Scoring assingment. The main idea is to decide if the customer who is looking for a loan can get it or not. We have a dataset of historical data for all the customers who either 'returned their loan back' or 'defaulted'. Here is a definition of a 'default' in context of risk-scoring:

**Credit default risk** – *The risk of loss arising from a debtor being unlikely to pay its loan obligations in full or the debtor is more than 90 days past due on any material credit obligation; default risk may impact all credit-sensitive transactions, including loans, securities and derivatives.*

We want to make a model that will for each new customer predict the probability of defaulting. This week is mainly focused on decision-trees and the ensemble methods, but I will also check what `LogisticRegression` has to tell about it. It never hurts.

Here is everything you need to know about this dataset:
https://github.com/gastonstat/CreditScoring

In [1]:
data_url = 'https://raw.githubusercontent.com/gastonstat/CreditScoring/master/CreditScoring.csv'

In [1]:
#!wget -P ./data $data_url

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('./data/CreditScoring.csv')
df.head()

,Status,Seniority,Home,Time,Age,Marital,Records,Job,Expenses,Income,Assets,Debt,Amount,Price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,1,0,1,36,26,1,1,1,46,107,0,0,310,910


In [5]:
#Convert columns to lowercase
df.columns = df.columns.str.lower()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4455 entries, 0 to 4454
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   status     4455 non-null   int64
 1   seniority  4455 non-null   int64
 2   home       4455 non-null   int64
 3   time       4455 non-null   int64
 4   age        4455 non-null   int64
 5   marital    4455 non-null   int64
 6   records    4455 non-null   int64
 7   job        4455 non-null   int64
 8   expenses   4455 non-null   int64
 9   income     4455 non-null   int64
 10  assets     4455 non-null   int64
 11  debt       4455 non-null   int64
 12  amount     4455 non-null   int64
 13  price      4455 non-null   int64
dtypes: int64(14)
memory usage: 487.4 KB


In [7]:
#Using R code in the GitHub repo to assint in converting values of categorical features to something understandab;e
#but I've c/p this from the course repo:

status_values = {
    1: 'ok',
    2: 'default',
    0: 'unk'
}

df.status = df.status.map(status_values)

home_values = {
    1: 'rent',
    2: 'owner',
    3: 'private',
    4: 'ignore',
    5: 'parents',
    6: 'other',
    0: 'unk'
}

df.home = df.home.map(home_values)

marital_values = {
    1: 'single',
    2: 'married',
    3: 'widow',
    4: 'separated',
    5: 'divorced',
    0: 'unk'
}

df.marital = df.marital.map(marital_values)

records_values = {
    1: 'no',
    2: 'yes',
    0: 'unk'
}

df.records = df.records.map(records_values)

job_values = {
    1: 'fixed',
    2: 'partime',
    3: 'freelance',
    4: 'others',
    0: 'unk'
}

df.job = df.job.map(job_values)


In [8]:
df.head()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,ok,9,rent,60,30,married,no,freelance,73,129,0,0,800,846
1,ok,17,rent,60,58,widow,no,fixed,48,131,0,0,1000,1658
2,default,10,owner,36,46,married,yes,freelance,90,200,3000,0,2000,2985
3,ok,0,rent,60,24,single,no,fixed,63,182,2500,0,900,1325
4,ok,0,rent,36,26,single,no,fixed,46,107,0,0,310,910


In [9]:
#The missing values (based on data description) are values of type 99999.0000
df.describe().round()

,seniority,time,age,expenses,income,assets,debt,amount,price
count,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0
mean,8.0,46.0,37.0,56.0,763317.0,1060341.0,404382.0,1039.0,1463.0
std,8.0,15.0,11.0,20.0,8703625.0,10217569.0,6344253.0,475.0,628.0
min,0.0,6.0,18.0,35.0,0.0,0.0,0.0,100.0,105.0
25%,2.0,36.0,28.0,35.0,80.0,0.0,0.0,700.0,1118.0
50%,5.0,48.0,36.0,51.0,120.0,3500.0,0.0,1000.0,1400.0
75%,12.0,60.0,45.0,72.0,166.0,6000.0,0.0,1300.0,1692.0
max,48.0,72.0,68.0,180.0,99999999.0,99999999.0,99999999.0,5000.0,11140.0


This is an useful trick. To find which one have missing values, you do describe and see the max values.
When you fix them to NaNs then "the next maximum value" will occur.

In [10]:
for c in ['income', 'assets', 'debt']:
    df[c] = df[c].replace(to_replace=99999999, value=np.nan)

In [11]:
df.describe().round()

,seniority,time,age,expenses,income,assets,debt,amount,price
count,4455.0,4455.0,4455.0,4455.0,4421.0,4408.0,4437.0,4455.0,4455.0
mean,8.0,46.0,37.0,56.0,131.0,5403.0,343.0,1039.0,1463.0
std,8.0,15.0,11.0,20.0,86.0,11573.0,1246.0,475.0,628.0
min,0.0,6.0,18.0,35.0,0.0,0.0,0.0,100.0,105.0
25%,2.0,36.0,28.0,35.0,80.0,0.0,0.0,700.0,1118.0
50%,5.0,48.0,36.0,51.0,120.0,3000.0,0.0,1000.0,1400.0
75%,12.0,60.0,45.0,72.0,165.0,6000.0,0.0,1300.0,1692.0
max,48.0,72.0,68.0,180.0,959.0,300000.0,30000.0,5000.0,11140.0


In [12]:
#Here is another clever trick to sift out that one 'unk' value for 'status'
df = df[df.status != 'unk'].reset_index(drop=True)

In [13]:
#Train/test split
from sklearn.model_selection import train_test_split

RANDOM_STATE = 42

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=RANDOM_STATE)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=RANDOM_STATE)

In [14]:
#Reseting the index of them since they've shuffeled so when you finally sift the y values you get no confusion
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [15]:
#The 'defaults' should be 1's because we are making a model to predict those who will potentialy 'default'
y_train = (df_train.status == 'default').astype('int').values
y_val = (df_val.status == 'default').astype('int').values
y_test = (df_test.status == 'default').astype('int').values

#Now that we collected all of them it is time to delete it from the feature matrix so that we don't introduce data-leaks
del df_train['status']
del df_val['status']
del df_test['status']

## Training and Evaluating the DecisionTree using scikit-learn

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score

In [17]:
dv = DictVectorizer(sparse=False)

In [18]:
#Let's see how many NaNs are in the whole dataset
df.isna().sum()

status        0
seniority     0
home          0
time          0
age           0
marital       0
records       0
job           0
expenses      0
income       34
assets       47
debt         18
amount        0
price         0
dtype: int64

In [19]:
#Precentage of NaNs in a dataset
round((df.isna().sum().sum()*100/len(df)),2)

2.22

DecisionTrees can't handle NaNs since they are assesing splits using Information Gain. At this point, instructor probably has a good reason (for now) that he filled those NaNs with 0. If you ask me, since it is only 2.22% of the dataset I would delete those rows. But, also wont exclude the possibility that I can use some clever back-of-the-envelope calculations to asses if I can fix them. Maybe, there is a possibility of making them stay NaNs. ;) 

In [20]:
#Since it is called Dict Vectorizer it means we have to convert the dataset into dicitionary
#before vectorizing it 

train_dicts = df_train.fillna(0).to_dict(orient='records')
val_dicts = df_val.fillna(0).to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)
X_valid = dv.transform(val_dicts)

In [21]:
#Let's train plain Decision Tree on the training dataset 
dt = DecisionTreeClassifier(random_state=RANDOM_STATE)
dt.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [22]:
#Now let's asses the AUC score on the y_val, but 

y_pred = dt.predict_proba(X_valid)[:, 1]
roc_auc_score(y_val, y_pred)

0.6510999347815181

At this point, you might be thinking 'such poor score it is just slightly better than Random Classifier!', but here there is a *trick* to explain it, and it will also explain why `predict_proba` gives you only ones and zeroes. 

Let's test the AUC score on the train dataset

In [23]:
#Let's test the AUC score on the train dataset
y_pred = dt.predict_proba(X_train)[:, 1]
roc_auc_score(y_train, y_pred)

0.9999996473061242

AUC of nearly 1? That is an ideal model! But the problem is that decision trees that have no 'prunning' will *always* overfit the training data, because for each instance in the dataset they will find sspecific rules (watch the intrudoctory video). 

It is not a problem of a model, since it wants to find the best score possible and in that it learns specific rules such that for each datapoint it gives perfect score. The more you have specific rules, the more you are good at memorizing the stuff rather than generalizing it. When you are memorizing each datapoint, yes it will be good at recognizing those, but how about new data? You can't just keep memorizing. That model is useless! You are basically training a model that for given input (dataset) will output perfect predictions only if your new data matches one of the rows in the dataset. That problem is called overfiting. 

So, after knowing that we are overfitting and for each datapoint we have a particular basket then why do predict proba gives 0s and 1s?

Decision trees are inherently non-probabilistic models. Unlike some other algorithms (e.g., logistic regression), decision trees don't naturally output probabilities. 

Instead `predict_proba` returns the fraction of samples of each class in the leaf node where a particular instance ends up. This is more of a distribution of class counts rather than true probabilities.

In [24]:
pd.DataFrame(dt.predict_proba(X_valid)).head()

,0,1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0


Let's make the tree grow up to depth of level 3 and see how good it 'generalizes'.

In [25]:
dt = DecisionTreeClassifier(max_depth=3, random_state=RANDOM_STATE)
dt.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, random_state=42)

In [26]:
y_pred = dt.predict_proba(X_train)[:, 1]
auc = roc_auc_score(y_train, y_pred)
print('train:', auc)

y_pred = dt.predict_proba(X_valid)[:, 1]
auc = roc_auc_score(y_val, y_pred)
print('val:', auc)

train: 0.774310483472765
val: 0.7455663974313953


Since there is no much of a difference between train and validation AUC score, we see that it generalizes pretty good. Let's see the rules it learned along the way:

In [27]:
from sklearn.tree import export_text
print(export_text(dt, feature_names=dv.feature_names_))

|--- seniority <= 2.50
|   |--- records=yes <= 0.50
|   |   |--- job=fixed <= 0.50
|   |   |   |--- class: 0
|   |   |--- job=fixed >  0.50
|   |   |   |--- class: 0
|   |--- records=yes >  0.50
|   |   |--- income <= 136.50
|   |   |   |--- class: 1
|   |   |--- income >  136.50
|   |   |   |--- class: 1
|--- seniority >  2.50
|   |--- records=yes <= 0.50
|   |   |--- income <= 93.50
|   |   |   |--- class: 0
|   |   |--- income >  93.50
|   |   |   |--- class: 0
|   |--- records=yes >  0.50
|   |   |--- assets <= 3450.00
|   |   |   |--- class: 1
|   |   |--- assets >  3450.00
|   |   |   |--- class: 0



Let's give `max_depth==2` for the sake of simplicity in understanding these 'if-else' decisions DT learned:

In [28]:
dt = DecisionTreeClassifier(max_depth=2, random_state=RANDOM_STATE)
dt.fit(X_train, y_train)

y_pred = dt.predict_proba(X_train)[:, 1]
auc = roc_auc_score(y_train, y_pred)
print('train:', auc)

y_pred = dt.predict_proba(X_valid)[:, 1]
auc = roc_auc_score(y_val, y_pred)
print('val:', auc)

train: 0.7175133670978937
val: 0.7128913108914865


In [29]:
print(export_text(dt, feature_names=dv.feature_names_))

|--- seniority <= 2.50
|   |--- records=yes <= 0.50
|   |   |--- class: 0
|   |--- records=yes >  0.50
|   |   |--- class: 1
|--- seniority >  2.50
|   |--- records=yes <= 0.50
|   |   |--- class: 0
|   |--- records=yes >  0.50
|   |   |--- class: 0



Out of all these let's focus on how would you interpret rules like `records=yes <= 0.50`.

`records` is a binary variable with values 'yes' and 'no':

In [30]:
df.records.value_counts()

records
no     3681
yes     773
Name: count, dtype: int64

This is the thing that might be confusing ex. `records=yes <= 0.5`. 

Given that `records=yes` is `True` and we map `True` to `1`, then `records=yes <= 0.5` would mean `True or 1 <=1` aka `records=yes and it is the opposite of that` (so effectively it is `records=no`), whilst `records=yes > 0.5` would mean `records=yes and it is right that`. 

--- 

At this point the Alexey is explaining the simple implementation of Decision Tree algorithm (this course is a refresher for me!):

- For each column name all the Thresholds.
- For each threshold split the column into two parts (left and right) and check impurity for left and right. For each 'side' if there are less elements of class A than class B and vice versa, then those are 'impurities' so we asses percentage of them. 
- We keep the impurity score for each of the threshold both for 'left' and 'right' split, and summarize them into one value (simple average is sufficient for this very simple implementation)
- After one column is finished we do the same process for the next column, and next and next etc. 
- Then we make the first split based on the best avg. impurity. 
- We are left with 'left' and 'right' sid'. Now, with less data and again with all the columns, for each side we recursively do the same process IF they are not pure, IF we have't reached `max_depth` and IF we have a limit of number of elements per class for each split (`max_samples_leaf`). 


For tuning the model Alexey uses a slightly economic variant of Grid Search. He trains Decision Tree for max depth, looks for ~5 values that give best scores and then given those values finds best `max_samples_leaf`. His approach is more visual and I do like it, but in a nutshell I would keep using GridSearch since DecisionTrees are fairly fast to train.  

Let's test the GridSearch.

In [31]:
max_depth_values = [2, 3, 4, 5, 6, 10, 15, 20]
max_samples_per_leaf_values = [1, 2, 5, 10, 15, 20, 50, 100, 200, 300] #The bigger values are there because of 

In [32]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': max_depth_values,
    'min_samples_leaf': max_samples_per_leaf_values
}

grid_search = GridSearchCV(dt, param_grid, cv=5, scoring='roc_auc')

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=2, random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5, 6, 10, 15, 20],
                         'min_samples_leaf': [1, 2, 5, 10, 15, 20, 50, 100, 200,
                                              300]},
             scoring='roc_auc')

In [33]:
#Print the best parameters and corresponding AUC score
print("Best Parameters: ", grid_search.best_params_)
print("Best AUC: {:.2f}".format(grid_search.best_score_))

Best Parameters:  {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 50}
Best AUC: 0.80


In [34]:
#Using the best model let's test the AUC score
best_model = grid_search.best_estimator_
test_auc = roc_auc_score(y_val, best_model.predict_proba(X_valid)[:, 1])
print("Test AUC: {:.2f}".format(test_auc))

Test AUC: 0.78


We see that it is not overfitting and that it gives decent AUC. Finally, lets train it on `df_full_train` and test it on `y_test`.

In [35]:
#Preparing the full_train
df_full_train = df_full_train.reset_index(drop=True)
y_full_train = (df_full_train.status == 'default').astype('int').values

del df_full_train['status']

full_train_dicts = df_full_train.fillna(0).to_dict(orient='records')
X_full_train = dv.transform(full_train_dicts)

In [36]:
#Training the moel on full_train
best_model.fit(X_full_train, y_full_train)

DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_leaf=50,
                       random_state=42)

In [37]:
#The test dataset is already prepared
test_dicts = df_test.fillna(0).to_dict(orient='records')
X_test = dv.transform(test_dicts)

In [38]:
test_auc = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])
print("Test AUC: {:.2f}".format(test_auc))

Test AUC: 0.80


As expected since we've trained on more data! Does the model predicts the probabilities?

In [39]:
best_model.predict_proba(X_test)

array([[0.50943396, 0.49056604],
       [1.        , 0.        ],
       [0.8627451 , 0.1372549 ],
       ...,
       [0.33333333, 0.66666667],
       [0.98245614, 0.01754386],
       [1.        , 0.        ]])

No, they look like probabilities but in an essence Decision Trees are not probabilistic models. Do they base any of their decision based on probability? Well, the Gini Impurity score is based on the probability but more or less thinking of it as 'impurity' to decide which feature has best treshold. 

Here is a quick reading: 
https://medium.com/ml-byte-size/how-does-decision-tree-output-predict-proba-12c78634c9d5


What is left in those leaves after training is proportion of 0s and 1s. Maybe could be interpreted as a proxy for probability. 

## Random Forest

In a nutshell: Instead of one tree lets have a board of members, different individuals who are basing their result on their 'unique' characteristics. If they have the same characteristics then the model would not be that interesting and why would someone hire 5 people with the same opinion if one would suffice? How to make decision trees more 'individual' in nature? Train them on different features (randomly chosen), possibly with different subsets of data to make them more 'individual'. Interestingly enough, by increasing variability we are getting more and more 'precise' so to speak. After everyone gives an opinion, we take an average. Since we take those 'proxies of probabilities' into account, then averaging them gives you 'true probability'. As for assesing any probability estimate the more you take into account when taking average, by the law of large numbers, the average would converge to 'true probability', and hence after having more trees than ex. 200 the result would keep stagnating. 

Again, I will use GridSearchCV to find the best model although I strongly recommend you if you didn't take first course in ML to watch the videos and go through the process like Alexey did it. You will gain a better understanding!

In [40]:
from sklearn.ensemble import RandomForestClassifier

In [41]:
rf = RandomForestClassifier(random_state=RANDOM_STATE)

param_grid = {
    'n_estimators' : [20, 50, 100, 200],
    'max_depth': max_depth_values,
    'min_samples_leaf': max_samples_per_leaf_values,
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False] #if False whole dataset is used to train each tree, and vice versa
}

grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='roc_auc')

In [42]:
#%timeit
#grid_search.fit(X_train, y_train)

In [43]:
#To not retrain it each time we start this ipynb uncomment the cell above. Here is what I've found:
best_model = RandomForestClassifier(max_depth=10, min_samples_leaf=50,
                       n_estimators=100, max_features='sqrt', random_state=RANDOM_STATE)

best_model.fit(X_full_train, y_full_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=50, random_state=42)

In [44]:
test_dicts = df_test.fillna(0).to_dict(orient='records')
X_test = dv.transform(test_dicts)

test_auc = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])
print("Test AUC: {:.2f}".format(test_auc))

Test AUC: 0.83


In [45]:
#Is it overfitting?

test_auc = roc_auc_score(y_val, best_model.predict_proba(X_valid)[:, 1])
print("Test AUC: {:.2f}".format(test_auc))

Test AUC: 0.85


Slightly better than pure Decision Tree. Finally let's train the XGBoost. 

## Gradient Boosted Trees (AnyBoost)

I felt an urge to revisit how gradient boosted trees work and Boosting in particular because I so much like the idea. No lectures, blogs, videos etc. gave me a click until I saw awesome lecture by amazing professor Kilian Q. Weinberger. If you are just a bit math inclined, and you have some math matiruity, and you have freshly in your head concepts of inner product, thinking of functions as vectors in n-dimensional space look no further than this:

https://www.youtube.com/watch?v=dosOtgSdbnY

Even if you don't know them well, you will possibly gain a good intuition. What I will present here is a bit 'decoded' version of the lecture, for my future reference, and what I think will be of much importance to understand it even better.

### Dot-product revisited

First, the professor is drawing a line (hyperplane) which is orthogonal to a vector. Why? There are two versions of interpreting dot product (inner product) for case of a 2D:

a) Given two vectors u and w, you want to find projection of u onto w. You draw a line through the vector w, and project the vector u orthogonally onto that line and calculate the the projection. 


\begin{equation}
\text{proj}_{\mathbf{w}}(\mathbf{u}) = \frac{\mathbf{u} \cdot \mathbf{w}}{\|\mathbf{w}\|^2} \cdot \mathbf{w}
\end{equation}

b) Given the same two vectors, you draw a line perpendicular to a vector u. Now you project vector w onto that line and the dot product becomes the usual (Polar) version:

\begin{equation}
\mathbf{u} \cdot \mathbf{w} = \|\mathbf{u}\| \cdot \|\mathbf{w}\| \cdot \cos(\theta)
\end{equation}


In both cases you get a scalar value, and that scalar value tells you if they are pointing in the same direction (dot_product>0), opposite direction (dot_product<0) or they are orthogonal to one another (dot_product=0). 

Refer to this if you feel a bit rusty on it: https://www.youtube.com/watch?v=LyGKycYT2v0


### They always converge. Why?

To give the intuition of 'why should boosting always converge towards y_true' he draws something like this:

<img src="./imgs/pythagoras_v2.png" width = "30%"></img>

Think of this 'sequential' stuff similar to when you walk in snow and leaving footprints for someone behind you to get his/her shoes less wet. Think of walking towards where each vector points to and stop there. Wait till new weak learner, which will give you 'new predictions', appears and then walk there etc.

Say that you already trained some weak learners and you get to some point which presents 'the latest predictions of latest weak learner' (red point). That is your starting point. Now, the whole idea (as I will later explain) is that from that point you train a hypothetical next weak learner such that the dot product between vector pointing towards y vector (which are true values of y) AND the vector (next weak learner which will lead you to new predictions) is bigger than 0. That is one of the reasons why it will eventually converge. The whole idea is to find such inner-product between those two vectors to minimize the loss. So we draw an arbitrary vector which satisfies that condition and presents 'next weak learner' or, should I say, 'updated predictions of next weak learner'. 

Given the 'present predictions of latest weak learner' and some arbitrary new weak learner which will give you 'next predictions' which satisfies the idea of pointing in the same direction as vector that points towards y_true we can draw a right triangle. Say we have a weak learner that overshoots to some point. We draw a right triangle, containing 'starting point', 'point which presents predictions of next weak learner' and the point that presents 'true labels'.

Since weak learner gives predictions which could potentially 'overshoot' (the vector has huge magnitude) we only trust it some percent (which is the learning rate!). Say you have a vector of magnitude 10. Multiply each component of a vector with some value <1 (which is learning rate) and the vector will shrink. 

The present hypotenuse (green one) is the magnitude of vector from the current weak learner to the true labels. If on the same triangle we 'move' towards blue point which represents 'hypothetical' new weak learner (vector that shinked using learning rate smaller than 1) then a^2 stays the same, b^2 is smaller and if we moved towards the blue point (next weak learner) then we would have 'new (updated) hypotenuse/purple one' which is smaller in magnitude than the previous one!

If we keep repeating the same process, satisfying the condition for dot (inner) product, then eventually we will converge towards y_true. Draw it on paper by satisfying the condition each step and you will see that you will always converge toward the point of y_true.

What if we have negative dot product? That means they are pointing in the opposite direction. If that is the case we just flip the vector and they will point to the same direction!

What if we have case when dot product is 0? Well, we can't do any tricks in that case. The vectors are orthogonal and the algorithm stops. Maybe nudge the vector in some direction and check the dot product?



---

The premise: Can weak learners (H) be combined to generate a strong learner with low bias. 

Here is some basic intuition that might guide you well whilst watching the video. Imagine an n-dimensional space where each axis represents labels of your training dataset. For the simplicity sake think in 3D. Think for a moment that the axis are real numbers but there is only one number which shows true label. Let's introduce the vector y_true which represents true label for each datapoint (each axis), be it binary classification or regression problem. Imagine that vector as one pointing to a datapoint in n-dimensional space. How would we get to that point by "combining" weak learners? We started with a premise that we want a lot of weak learners to be combined to generate a strong one. 

Idea of combining might be at first guess something to do with "adding", and we will see later why it is 'sequential'. When you have a very weak learner (something slightly better than random guessing) it will obviously predict something bad and not at all close to true values. Imagine it as a decision stump. So, you train a model and it produces some y_predicted values. Now, try to visualize the vector having values of y_predicted. You don't expect from weak learner to be 'perfect' so vector y_predicted and y_true are not close, maybe they even point out at different directions. 

What matters is the idea of 'trusting those predictions' which is our learning rate or `eta`. If you have a vector and multiply it with a scalar value <1 you will see that it's magnitude will change and it will 'shrink' (sometimes learning rate is called 'shrinkage'). Starting from that how do we find next tree? First of all remember that each time you find a predictions, that very datapoint will serve us as a center of a NEW coordinate system. That will be our 'new starting point' (refer to the 'footsteps in a snow from the above').

To find the next weak learner (we already trained one!) we expect from this starting point to have a next one such that the loss between new weak learner's predictions and true values are small. In some sense we want to 'guide' each new weak learners prediction vector towards the actual values. Here is where a math gets little bit messy, but if you watched the video till the end you would find out that loss presents vector that points out from true values to our starting point. Basically we need to minimize the inner product between vector H(xi)-y_true (where H(xi) are all the week learners we added up until this moment) AND h(xi) the weak learner we want to train. 

Obviously, the vector H(xi)-y_true points towards our starting point because that is the idea of vector subtraction. So, in order to minimize the inner-product it has to be more and more 'negative'. Given our 'starting point' and the vector pointing towards the starting point, where should our next vector point to? 
Remember that dot product is negative if they point in opposite directions? Well, to make inner-product negative they have to point in the opposite direction so the 'new' vector should always point on the opposite of the vector H(xi)-y_true to keep inner-product more negative. 

How I think that algorithm works internaly at this point: given all the combinations of features and thresholds to split it and make a decision tree, find the weak learner that gives predictions such that inner-product between those predictions and the 'compass' vector H(xi)-y_true is minimum. 

---


After a while you will see that this kind of pattern reminds you of 'gradient descent'. Say you have a random function. If you don't have any objective in mind then when you take a gradient of a function at some random point it will point you to something (either maximum or minimum). But if the objective is to minimize something it does not necessarily means that it has to be zero. In both cases be it finding minima or 'the most negative point' you are doing a bunch of steps. 

Oh, and the 'gradient' in gradient boosting refers to residual. If you ever seen linear regression then you will see that residual is defined as `actual-predicted` which is same as H(xi)-y. 

If the learning rate is big there is a high chance of overshooting or that algo keeps diverging instead of converging towards y_true. 
If the learning rate is small then we are taking baby-steps but maybe we stop in the middle of the road towards y_true? Then we need more weak learners (increase the number of boosting rounds). 

---

What I will do next is train plain XGBoost to see if everything is set properly, but I will not do GridSearchCV because there are too much parameters to learn and the GridSearchCV started to feel a little dull to use. Imagine how much time it would take to test different permutations for GBT's. Knowing that such problem is arising, the course instructor suggested me to look for the MLFlow in the MLOps Zoomcamp. I will train the basic model here and use the best parameters that instructor obtained. Then I will detach to learn a bit about MLFlow and then come back to this course. 